In [1]:
import numpy as np
import pandas as pd
import random
import torch
import os
import spacy
from torchtext.vocab import GloVe, FastText
from torch.utils.data.sampler import SubsetRandomSampler
from torch.utils.data import DataLoader
from sklearn.metrics import classification_report, accuracy_score
import torch.nn as nn
import torch.nn.functional as F
from tqdm import tqdm
import nltk
import matplotlib.pyplot as plt

2023-08-17 22:35:20.789646: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-08-17 22:35:21.280877: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
2023-08-17 22:35:21.747865: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:995] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-08-17 22:35:21.748121: W tensorflow/core/common_runtime/gpu/gpu_device.cc:1960] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GP

In [2]:
# Different Classes and their counts
data = pd.read_excel('/data/home/ayyoobmohd/DLNLP/Glove-and-Sentiments/Datasets/ClassificationDataset0.xlsx')
print(data["Label"].value_counts())
print("--------------------------------------------------------")
size = len(data["Review"])
print(f"Number of reviews here in dataset are : {size}")

neutral     2588
positive    1287
negative     125
Name: Label, dtype: int64
--------------------------------------------------------
Number of reviews here in dataset are : 4000


In [3]:
csv_data_path = '/data/home/ayyoobmohd/DLNLP/Glove-and-Sentiments/data/Dataset0.csv'

# def making_new_dataset(data):
#     data.to_csv(pos_path, index = False, header= False,
#           encoding = "latin-1", columns = ['Positive Review'])
    
#     positive_set = open(pos_path, "r", encoding="latin-1").read()
#     negative_set = open(neg_path, "r", encoding="latin-1").read()
    
#     pos_set = positive_set.split("\n")[:-1]
#     neg_set = negative_set.split("\n")[:-1]
    
#     #print(len(positive_data), len(negative_data))
    
#     return pos_set, neg_set

# positive_data, negative_data = making_new_dataset(data) 
csv = data.to_csv(csv_data_path, index = False, header= True)

In [4]:
#library that contains punctuation
import string
string.punctuation

#defining the function to remove punctuation
def remove_punctuation(text):
    punctuationfree="".join([i for i in text if i not in string.punctuation])
    return punctuationfree

In [5]:
import csv

pos_set = []
neg_set = []
neutral_set = []
with open("/data/home/ayyoobmohd/DLNLP/Glove-and-Sentiments/data/Dataset0.csv", encoding='utf-8') as csvf:
    data = csv.DictReader(csvf)
    #data['Review'] = data['Review'].apply(lambda x:remove_punctuation(x))
    for rows in data:
        # Removing punctuations
        chars_to_remove = [ '+', '#', '¡', '§', '…','‘', '’', '¿', '«', '»', '¨', '%', '-', '“', '”', '--', '`', '~', '<', '>', '*', '{', '}', '^', '=', '_', '[', ']', '|', '- ', '/']
        
        review = rows['Review'].replace('<br />', " ", -1)
        review = review.replace('´', "'", -1)
        for char in chars_to_remove:
            review = review.replace(char, " ", -1)
        
        
        if rows['Label'] == 'positive':
            pos_set.append(rows['Review'])
        elif rows['Label'] == 'negative':
            neg_set.append(rows['Review'])
        else:
            neutral_set.append(rows['Review'])

In [6]:
from torchtext.data import get_tokenizer
global_vectors = GloVe(name='840B', dim=300)

def preprocess_tokenized_reviewText_dataset0(pos, neg, neutral):
    
    nlp = spacy.load("en_core_web_md")
    data_set = []
    vocab = []
    chars_to_remove = ['--', '`', '~', '<', '>', '*', '{', '}', '^', '=', '_', '[', ']', '|', '- ', '.', ',']
    tokenizer = get_tokenizer("basic_english")
    
    for line in pos:
        # Tokenizes the input text into words
        tokens = tokenizer(line)

        data_set.append((tokens, 1))
        # Adds the extracted words to a list
        vocab.extend(tokens)
    print(f"--- Positive with label 1 ---- Finished ---")
    
    for line in neg:
        # Tokenizes the input text into words
        tokens = tokenizer(line)

        data_set.append((tokens, 0))
        # Adds the extracted words to a list
        vocab.extend(tokens)
    print(f"--- Negative with label 0 ----  Finished ---")
    
    for line in neutral:
        # Tokenizes the input text into words
        tokens = tokenizer(line)

        data_set.append((tokens, 2))
        # Adds the extracted words to a list
        vocab.extend(tokens)
    print(f"--- Neutral with label 2 ---- Finished ---")
    
    # Stores all the unique words in the dataset and their frequencies
    vocabulary = {}

    # Calculates the frequency of each unique word in the vocabulary
    for word in vocab:
        if word in vocabulary:
            vocabulary[word] += 1
        else:
            vocabulary[word] = 1

    print("Number of unique words in the vocabulary: ", len(vocabulary))
    return data_set, vocabulary

def sort_key(s):
    return len(s[0])

In [7]:
data, vocabulary = preprocess_tokenized_reviewText_dataset0(pos_set, neg_set, neutral_set)
print(f"Length of Complete concatenated Dataset of reviews : {len(data)}")

--- Positive with label 1 ---- Finished ---
--- Negative with label 0 ----  Finished ---
--- Neutral with label 2 ---- Finished ---
Number of unique words in the vocabulary:  9542
Length of Complete concatenated Dataset of reviews : 4000


In [8]:
# with open('temp.txt', 'w') as fp:
#     fp.write('\n'.join('%s %s' % x for x in data))


In [9]:
# Tokenization function
def tokenize(corpus, ids_vocab):
    """
        Converts words in the dataset to integer tokens
    """

    tokenized_corpus = []
    for line, sentiment in corpus:
        new_line = []
        for i, word in enumerate(line):
            if word in ids_vocab and (i == 0 or word != line[i-1]):
                new_line.append(ids_vocab[word])

        new_line = torch.Tensor(new_line).long()
        tokenized_corpus.append((new_line, sentiment))

    return tokenized_corpus

def assign_ids_to_words(vocab):
    # Stores the integer token for each unique word in the vocabulary
    ids_vocab = {}

    id = 0
    # Assigns words in the vocabulary to integer tokens
    for word, v in vocabulary.items():
        ids_vocab[word] = id
        id += 1
    
    return ids_vocab

ids_vocab = assign_ids_to_words(vocabulary)
print(len(ids_vocab))
#print(ids_vocab.items())

9542


In [10]:
token_corpus = tokenize(data, ids_vocab)
#token_corpus_test = tokenize(test_set, ids_vocab)

In [11]:
# Loading the embedding matrix
emb_dim = 300

embeds = torch.zeros(len(ids_vocab) + 1, emb_dim)

n = 0
for token, idx in ids_vocab.items():
    embeds[idx] = global_vectors[token]

    if sum(embeds[idx]) == 0:
        embeds[idx] = torch.rand(300)
        n+=1
print(n)

1739


In [12]:
embeds.shape

torch.Size([9543, 300])

In [13]:
def set_seed(seed = 42):
    '''
        For Reproducibility: Sets the seed of the entire notebook.
    '''

    np.random.seed(seed)
    random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    
    # When running on the CuDNN backend, two further options must be set
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = True
    
    # Sets a fixed value for the hash seed
    os.environ['PYTHONHASHSEED'] = str(seed)
    
def split_indices_tokenized(num_values, percentage):

    # Determine size of Validation set
    val_size = int(percentage * num_values)

    # Create random permutation of 0 to num_values-1
    idxs = np.random.permutation(num_values)
    return np.sort(idxs[val_size:]), np.sort(idxs[:val_size])

set_seed(1)
train_pos_indices, val_pos_indices = split_indices_tokenized(len(pos_set), 0.1)
train_neg_indices, val_neg_indices = split_indices_tokenized(len(neg_set), 0.1)
train_neutral_indices, val_neutral_indices = split_indices_tokenized(len(neutral_set), 0.1)
#+len(pos_set)-1
train_indices = np.concatenate((train_pos_indices, train_neg_indices, train_neutral_indices))
val_indices = np.concatenate((val_pos_indices, val_neg_indices, train_neutral_indices))

In [14]:
len(train_indices), len(val_indices)

(3602, 2470)

In [15]:
from torch.nn.utils.rnn import pad_sequence

# ----------- Batching the data -----------
def collate_fn(instn):

    sentence = [x[0] for x in instn]

    # Pre padding
    sen_len = [len(x[0]) for x in instn]
    max_len = max(sen_len)

    padded_sent = torch.zeros(1, max_len)
    sentence_pad = [torch.cat((torch.zeros(max_len-len(x[0])), x[0]), dim=0) for x in instn]
    
    for i in sentence_pad:
        padded_sent = torch.cat((padded_sent, i.unsqueeze(dim=0)), dim=0)
    padded_sent = padded_sent[1:].long()

    # Post padding
    #padded_sent = pad_sequence(sentence, batch_first=True, padding_value=0)

    labels = torch.Tensor([x[1] for x in instn])

    return (padded_sent, labels)


batch_size = 64

train_sampler   = SubsetRandomSampler(train_indices)
train_loader    = DataLoader(token_corpus, batch_size, sampler=train_sampler, collate_fn=collate_fn)

val_sampler     = SubsetRandomSampler(val_indices)
val_loader      = DataLoader(token_corpus, batch_size, sampler=val_sampler, collate_fn=collate_fn)

In [16]:
device = "cuda" if torch.cuda.is_available() else "cpu"

## DAN

In [137]:
class DAN(nn.Module):
    
    def __init__(self):
        super().__init__()

        self.linear1 = nn.Linear(600, 1024)
        self.linear2 = nn.Linear(1024, 2048)
        self.linear3 = nn.Linear(2048, 512)
        self.linear4 = nn.Linear(512, 64)
        self.linear5 = nn.Linear(64, 1)

    def forward(self, Xb, tsne = False):
        x = self.linear1(Xb)
        x = F.relu(x)
        x = self.linear2(x)
        x = F.relu(x) 
        x = self.linear3(x)
        x = F.relu(x)
        x = self.linear4(x)
        
        if tsne == True:
            return x
            
        x = F.relu(x)
        x = self.linear5(x)
        x = torch.sigmoid(x)
        return x
    
# ----------- Model -----------
class BILSTM(nn.Module):
    
    def __init__(self, embeds):
        super().__init__()

        self.embeddings = nn.Embedding.from_pretrained(embeds, padding_idx=0)

        self.gru = nn.GRU(input_size = 300, hidden_size = 128, num_layers = 2, batch_first = True, bidirectional = True, dropout=0.5)

        self.lin1 = nn.Linear(256, 64)
        self.lin2 = nn.Linear(64, 3)

        self.lin3 = nn.Linear(256, 1)

    def forward(self, xb):

        xe = self.embeddings(xb)
        out, y = self.gru(xe)
        
        x = self.lin3(out).squeeze(dim=-1)
        x = torch.softmax(x, dim=-1).unsqueeze(dim=1)
        x = torch.bmm(x, out).squeeze(dim=1)              # Weighted average

#         #x = torch.cat((x, y[2][ :, :], y[3][ :, :]), dim = 1) # Tried concatenating the representation with hidden units - got similar results
        x = self.lin1(x)
        x = F.relu(x)
        x = self.lin2(x)
        x = torch.softmax(x, dim=1)
        return x

In [145]:
for xb, yb in train_loader:
    eg = xb
    eg_y = yb
    break
eg_y = eg_y.type(torch.LongTensor)

In [146]:
model1 = BILSTM(embeds)
yi = model1(eg.type(torch.LongTensor))
print(yi.shape)
yi, eg_y

torch.Size([64, 3])


(tensor([[0.3470, 0.3366, 0.3164],
         [0.3459, 0.3353, 0.3187],
         [0.3423, 0.3387, 0.3190],
         [0.3398, 0.3399, 0.3202],
         [0.3375, 0.3388, 0.3237],
         [0.3448, 0.3351, 0.3201],
         [0.3406, 0.3398, 0.3196],
         [0.3422, 0.3403, 0.3175],
         [0.3520, 0.3358, 0.3122],
         [0.3469, 0.3342, 0.3189],
         [0.3389, 0.3385, 0.3225],
         [0.3399, 0.3412, 0.3189],
         [0.3439, 0.3356, 0.3206],
         [0.3404, 0.3366, 0.3230],
         [0.3437, 0.3316, 0.3248],
         [0.3441, 0.3361, 0.3198],
         [0.3449, 0.3369, 0.3182],
         [0.3403, 0.3412, 0.3184],
         [0.3428, 0.3372, 0.3201],
         [0.3413, 0.3353, 0.3234],
         [0.3423, 0.3374, 0.3203],
         [0.3456, 0.3368, 0.3176],
         [0.3396, 0.3374, 0.3230],
         [0.3411, 0.3437, 0.3152],
         [0.3420, 0.3369, 0.3212],
         [0.3444, 0.3349, 0.3207],
         [0.3413, 0.3370, 0.3217],
         [0.3435, 0.3346, 0.3220],
         [0.3429, 0.

In [148]:
loss = loss_fn_c(yi, eg_y)
loss

tensor(1.1001, grad_fn=<NllLossBackward0>)

In [144]:
model = BILSTM(embeds)
model.to(device)
opt_c = torch.optim.Adam(model.parameters(), lr = 0.001) # Same as Adam with weight decay = 0.001
loss_fn_c = F.cross_entropy

In [115]:
model1 = BILSTM(embeds)
yi = model1(eg)


In [116]:
from sklearn.metrics import f1_score

In [118]:
# ----------- Main Training Loop -----------
max_epoch = 10

best_test_acc = 0
for ep in range(max_epoch):

    epoch_loss = 0

    model.train()

    for xb, yb in tqdm(train_loader):
        yb = yb.type(torch.LongTensor)
        xb = xb.to(device)
        yb = yb.to(device)

        y_hat = model(xb)
        loss = loss_fn_c(y_hat.squeeze(), yb)

        loss.backward()

        opt_c.step()

        opt_c.zero_grad()

        nn.utils.clip_grad_norm_(model.parameters(), 5)

        epoch_loss += float(loss)

    print("Epoch: ", ep+1, " Training Loss: ", epoch_loss/len(train_loader))


    #----------- Validation -----------

    val_labels = []
    val_pred = []

    model.eval()

    val_epoch_loss = 0

    with torch.no_grad():
        for xb, yb in tqdm(val_loader):
            yb = yb.type(torch.LongTensor)
            xb = xb.to(device)
            yb = yb.to(device)

            y_hat = model(xb)
            loss = loss_fn_c(y_hat.squeeze(), yb)

            val_epoch_loss += float(loss)

            val_labels.extend(torch.round(yb).cpu().detach().numpy())
            val_pred.extend(y_hat.round().cpu().detach().numpy())

    print("Validation loss: ", val_epoch_loss/len(val_loader))
    print("Validation accuracy: ", accuracy_score(val_labels, val_pred)*100)

    if ep > 5 and prev_val_loss - val_epoch_loss > 0.015:
        print("Saving Model")
        torch.save(model.state_dict(), "GRU_Model_Dataset0.pt")
    
    prev_val_loss = val_epoch_loss

100%|██████████████████████████████████████████████████████████████████████████████████████████████| 57/57 [00:00<00:00, 347.19it/s]


Epoch:  1  Training Loss:  0.871560581943445


100%|██████████████████████████████████████████████████████████████████████████████████████████████| 39/39 [00:00<00:00, 675.60it/s]

Validation loss:  1.0177391370137532


ValueError: Classification metrics can't handle a mix of multiclass and multilabel-indicator targets